In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.model_selection import cross_validate
import tensorflow as tf
from sklearn.metrics import mean_squared_error as MSE
from pathlib import Path
from sklearn.metrics import classification_report
import xgboost as xg 
from xgboost import XGBClassifier
import pandas as pd , numpy as np , datetime, os
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras import layers
import seaborn as sns,  matplotlib.pyplot as plt, joblib as jlib
import tensorflow as tf, keras_tuner, tensorflow.keras as keras
from tensorflow.keras import layers



In [2]:
pip install csv-to-sqlite

Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv_to_sqlite

In [4]:
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="UTF-8")
#change path to "Resources/insurance.csv" for jupyter notebook
input_files = ['Resources/insurance.csv']
#change path to "Resources/insurance.sqlite" for jupyter notebook
csv_to_sqlite.write_csv(input_files, "Resources/insurance.sqlite", options)


Error on table Resources/insurance.csv: 
 table [insurance] already exists
Written 0 rows into 1 tables in 0.006 seconds


0

In [5]:
import sqlalchemy
from sqlalchemy import create_engine, text, inspect, func
import warnings
engine = create_engine("sqlite:///Resources/insurance.sqlite")

#gives warning but easier to get the dataframe from querying
warnings.filterwarnings('ignore')
conn = engine.raw_connection()
insurance_df = pd.read_sql_query(sql=f"SELECT age ,sex, bmi, children, smoker , region , ROUND(charges,2) as charges FROM insurance", con=conn)

In [6]:
insurance_df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92
1,18,male,33.770,1,no,southeast,1725.55
2,28,male,33.000,3,no,southeast,4449.46
3,33,male,22.705,0,no,northwest,21984.47
4,32,male,28.880,0,no,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.55
1334,18,female,31.920,0,no,northeast,2205.98
1335,18,female,36.850,0,no,southeast,1629.83
1336,21,female,25.800,0,no,southwest,2007.95


In [7]:
insurance_df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [8]:
# Define features set
X = insurance_df.copy()
X.drop("charges", axis=1, inplace=True)
X.head()

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest


In [9]:
# Define target vector
y = insurance_df["charges"].values.reshape(-1, 1)
y[:5]

array([[16884.92],
       [ 1725.55],
       [ 4449.46],
       [21984.47],
       [ 3866.86]])

In [10]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(X)

In [11]:
X[:5]

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,0,0,1,0,0,0,1
1,18,33.770,1,0,1,1,0,0,0,1,0
2,28,33.000,3,0,1,1,0,0,0,1,0
3,33,22.705,0,0,1,1,0,0,1,0,0
4,32,28.880,0,0,1,1,0,0,1,0,0


In [12]:
# # Splitting into Train and Test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.3, random_state = 123) 

In [13]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [14]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(train_X)

In [15]:
# Scale the training data
X_train_scaled = X_scaler.transform(train_X)
X_test_scaled = X_scaler.transform(test_X)

# XGboost

In [45]:
# # Instantiation 
# #xgb_r  = GridSearchCV(
#             estimator=xg.XGBRegressor(),
#             param_grid={"learning_rate": (0.05, 0.10, 0.15),
#                         "max_depth": [ 3, 4, 5, 6, 8, None],
#                         "min_child_weight": [ 1, 3, 5, 7],
#                         "gamma":[ 0.0, 0.1, 0.2],
#                         "colsample_bytree":[ 0.3, 0.4, 0.8]},
#              scoring='neg_mean_absolute_error', verbose=3, n_jobs=-1)

In [46]:
# Instantiation 
xgb_r  = GridSearchCV(
            estimator=xg.XGBRegressor(),
            param_grid={"learning_rate": (0.05, 0.10, 0.15),
                        "n_estimators" : (100, 200, 300, 500),
                        "max_depth": [ 3, 4, 5, 6, 8, None],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4, 0.8]},
             scoring='neg_mean_absolute_error', verbose=3, n_jobs=-1)

grid_result = MultiOutputRegressor(xgb_r).fit(train_X, train_y)
best_params = grid_result.best_params_

Fitting 5 folds for each of 2592 candidates, totalling 12960 fits
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=100;, score=-3398.592 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=200;, score=-2888.034 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2892.330 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=200;, score=-2893.676 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=500;, score=-2898.642 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=100;, score=-3534.205 total time=   0.0s
[CV 5/5] END col

[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=100;, score=-3526.159 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=300;, score=-2918.556 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2849.073 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=300;, score=-2796.691 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=100;, score=-3276.465 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=200;, score=-2802.244 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min

[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=200;, score=-2845.772 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2862.069 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=200;, score=-3088.564 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=500;, score=-2709.890 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=200;, score=-2870.451 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=7, n_estimators=100;, score=-3394.884 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min

[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=100;, score=-3677.262 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=300;, score=-2812.419 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=100;, score=-3277.284 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=100;, score=-3669.150 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=200;, score=-2844.218 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=300;, score=-2981.500 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min

[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=100;, score=-3691.409 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=200;, score=-3093.943 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2738.070 total time=   0.1s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=500;, score=-2846.583 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=200;, score=-2885.833 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=200;, score=-3089.094 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min

...

[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=100;, score=-3101.338 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=100;, score=-3045.483 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=100;, score=-3186.467 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=200;, score=-2918.020 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=200;, score=-2877.107 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=200;, score=-2857.299 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min

[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=100;, score=-3033.557 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=100;, score=-3095.561 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=100;, score=-3022.614 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=100;, score=-3159.009 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=200;, score=-2961.291 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=200;, score=-2873.026 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min

[CV 4/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=4, min_child_weight=7, n_estimators=100;, score=-2827.420 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, n_estimators=100;, score=-3266.722 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, n_estimators=100;, score=-3024.116 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, n_estimators=100;, score=-3226.854 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, n_estimators=100;, score=-2960.373 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=5, min_child_weight=5, n_estimators=100;, score=-3025.472 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=5, min

[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=500;, score=-2799.755 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=500;, score=-2998.146 total time=   0.1s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=500;, score=-2956.192 total time=   0.1s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=500;, score=-2834.048 total time=   0.1s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=500;, score=-2904.780 total time=   0.1s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=7, n_estimators=100;, score=-3218.913 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min

AttributeError: 'MultiOutputRegressor' object has no attribute 'best_params_'

In [47]:
xgb_r.fit(train_X, train_y)

Fitting 5 folds for each of 2592 candidates, totalling 12960 fits
[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=None, min_child_weight=5, n_estimators=100;, score=-3160.533 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=None, min_child_weight=5, n_estimators=100;, score=-3108.968 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=None, min_child_weight=5, n_estimators=100;, score=-3029.424 total time=   0.0s
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=None, min_child_weight=5, n_estimators=100;, score=-2883.768 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=None, min_child_weight=5, n_estimators=100;, score=-2813.647 total time=   0.0s
[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=None, min_child_weight=5, n_estimators=200;, score=-3358.687 total time=   0.0s
[CV 

[CV 2/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=6, min_child_weight=3, n_estimators=500;, score=-3686.344 total time=   0.1s
[CV 3/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=6, min_child_weight=3, n_estimators=500;, score=-3656.776 total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=6, min_child_weight=3, n_estimators=500;, score=-3522.420 total time=   0.1s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=6, min_child_weight=3, n_estimators=500;, score=-3569.911 total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min_child_weight=5, n_estimators=200;, score=-3426.716 total time=   0.1s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min_child_weight=5, n_estimators=200;, score=-3324.308 total time=   0.1s
[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min

[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=4, min_child_weight=5, n_estimators=500;, score=-3457.290 total time=   0.1s
[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=4, min_child_weight=7, n_estimators=100;, score=-2977.998 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=4, min_child_weight=7, n_estimators=100;, score=-2924.159 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=4, min_child_weight=7, n_estimators=100;, score=-2883.920 total time=   0.0s
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=4, min_child_weight=7, n_estimators=100;, score=-2660.471 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=5, min_child_weight=3, n_estimators=100;, score=-2944.717 total time=   0.0s
[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=5, min

[CV 2/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min_child_weight=1, n_estimators=500;, score=-3552.194 total time=   0.2s
[CV 3/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min_child_weight=1, n_estimators=500;, score=-3274.922 total time=   0.2s
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min_child_weight=1, n_estimators=500;, score=-3376.792 total time=   0.2s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min_child_weight=1, n_estimators=500;, score=-3204.974 total time=   0.2s
[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min_child_weight=3, n_estimators=100;, score=-3465.586 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min_child_weight=3, n_estimators=100;, score=-3461.291 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=8, min

[CV 1/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=None, min_child_weight=7, n_estimators=200;, score=-3029.476 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=None, min_child_weight=7, n_estimators=200;, score=-3119.162 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=None, min_child_weight=7, n_estimators=200;, score=-3001.566 total time=   0.0s
[CV 4/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=None, min_child_weight=7, n_estimators=200;, score=-2758.387 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=None, min_child_weight=7, n_estimators=200;, score=-2879.693 total time=   0.0s
[CV 1/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=None, min_child_weight=7, n_estimators=300;, score=-3136.057 total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.05

...

[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=100;, score=-2801.206 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=100;, score=-2871.440 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=100;, score=-2746.553 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=100;, score=-2874.788 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=200;, score=-3087.604 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=200;, score=-2897.150 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=4, min_child_

[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=7, n_estimators=100;, score=-2972.146 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=3, n_estimators=100;, score=-3297.995 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=3, n_estimators=200;, score=-3795.756 total time=   0.1s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=3, n_estimators=200;, score=-3632.225 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=3, n_estimators=200;, score=-3562.920 total time=   0.1s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=3, n_estimators=200;, score=-3316.719 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_

[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=3, n_estimators=100;, score=-3286.936 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=3, n_estimators=100;, score=-3146.798 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=3, n_estimators=100;, score=-2999.093 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=1, n_estimators=500;, score=-3647.967 total time=   0.1s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=1, n_estimators=500;, score=-3805.276 total time=   0.1s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=1, n_estimators=500;, score=-3400.439 total time=   0.1s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=6, min

[CV 3/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=8, min_child_weight=7, n_estimators=300;, score=-3521.384 total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=8, min_child_weight=7, n_estimators=300;, score=-3217.417 total time=   0.1s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=8, min_child_weight=7, n_estimators=300;, score=-3348.620 total time=   0.1s
[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=8, min_child_weight=7, n_estimators=500;, score=-3598.421 total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=8, min_child_weight=7, n_estimators=500;, score=-3581.449 total time=   0.1s
[CV 3/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=8, min_child_weight=7, n_estimators=500;, score=-3710.831 total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=8, min_child_

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bi...
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.3, 0.4, 0.8],
                         'gamma': [0.0, 0.1, 0.2],
                         'learning_rate': (0.05, 0.1, 0.15),
                         'max_depth': [3, 4, 5, 6, 8, None],
                         'min_child_weight': [1, 3, 5, 7],
                         'n_estimators': (100, 200, 300, 500)},
             scoring='neg_mean_absolute_error', verbose=3)

In [48]:
xgb_r.best_score_

-2619.405120796026

In [49]:
best_param = xgb_r.best_params_
best_param

{'colsample_bytree': 0.8,
 'gamma': 0.0,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 100}

In [50]:
# now we take the best possible params and build the model for training
xgboost_model = xg.XGBRegressor(n_estimators = best_param['n_estimators'],
                                     colsample_bytree=best_param['colsample_bytree'],
                                     learning_rate = best_param['learning_rate'],
                                     max_depth = best_param['max_depth'],
                                     min_child_weight=best_param['min_child_weight'])

In [51]:
# Fitting the model 
xgboost_model.fit(train_X, train_y) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [52]:
pred = xgb_r.predict(test_X)

In [53]:
# RMSE Computation 
rmse = np.sqrt(MSE(test_y, pred)) 
print("RMSE : % f" %(rmse)) 

RMSE :  4109.559166


In [54]:
xval_score = cross_validate(xgboost_model,
                            train_X, train_y,
                            scoring = ['r2', 'explained_variance'], verbose=3,
                            return_train_score=True, n_jobs=-1)
xval_score


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


{'fit_time': array([0.01239109, 0.00926685, 0.00900698, 0.00902796, 0.01336622]),
 'score_time': array([0.00129318, 0.00233221, 0.00115895, 0.00159097, 0.00117278]),
 'test_r2': array([0.85788655, 0.78939987, 0.84906951, 0.86501178, 0.85169605]),
 'train_r2': array([0.8920871 , 0.90111757, 0.89534063, 0.89181672, 0.89389724]),
 'test_explained_variance': array([0.85791833, 0.78965495, 0.84951208, 0.86502816, 0.85170727]),
 'train_explained_variance': array([0.89208742, 0.9011179 , 0.89534078, 0.89181696, 0.89389748])}

In [55]:
jlib.dump(xgboost_model, "xgboost_modelinsurance")

['xgboost_modelinsurance']